<h1>Pastebin Webscraping: Content</h1>

<h2>Importing libraries</h2>

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import mysql.connector
import getpass

In [2]:
pswd = getpass.getpass('Password:')

Password:········


*If you ever needed to re-create the table...*

try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebincontent (LINK varchar(40), TEXT varchar(60000))")
except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()

<h2>Defining the Webscraping Spider</h2>

In [3]:
class MySpider(scrapy.Spider):
    name = "Pastebin"

    custom_settings = {
        'DOWNLOAD_DELAY': 2,
        'RETRY_TIMES' : 3
    }
    
    def start_requests(self):
        
        while 1 == 1:
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                cursor.execute("SELECT * FROM pastebin")
                result = cursor.fetchall()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()

            for link in result:
                try:
                    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                    cursor = cnx.cursor(buffered=True)
                    query = "DELETE FROM pastebin WHERE LINK = '"+link[0]+"'"
                    cursor.execute(query)
                    cnx.commit()
                except mysql.connector.Error as err:
                    print(str(err))
                else:
                    cursor.close()
                    cnx.close()

                yield scrapy.Request(url="https://pastebin.com"+link[0].encode('ascii','ignore'), callback=self.parse, dont_filter=True)
        
    def parse(self, response):
        targets = ["gsi.gov.uk","spotify","netflix","steam key","minecraft nfa","deezer","crunchyroll","mcnfa","hulu","hma key","fortnite","nordvpn"]
        content = response.xpath('//textarea//text()').extract()[0]
        
        if any(x in content.lower() for x in targets):
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                query = "INSERT INTO pastebincontent (LINK, TEXT) VALUES ('"+response.request.url+"', '"+content.encode('hex')+"')"
                cursor.execute(query)
                cnx.commit()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()

<h2>Running the Webscraping</h2>

Note, you can't re run the code below in a single session for one reason or another, so you need to restart the kernel between runs.

This code creates a lightweight container for our webspider and then runs it - to be honest understanding this is probably optional unless it breaks.

In [ ]:
process = CrawlerProcess()
process.crawl(MySpider)
process.start()

2018-05-01 18:40:41 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2018-05-01 18:40:41 [scrapy.utils.log] INFO: Overridden settings: {}
2018-05-01 18:40:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-05-01 18:40:41 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddle